<a href="https://colab.research.google.com/github/aa3025/youtube/blob/main/Youtube_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Instructions:**
1. Paste Playlist URL from Youtube to "url:" field
2. If you dont need the whole playlist, enter numbers into "playlist_start" and "playlist_end" fields and untick "download_full_playlist"
3. If you want to download videos, tick "download_videos", otherwise only audio MP3 will be saved.
4. Enter text to "playlist_album" and "playlist_artist" to tag MP3 files after downloading.
5. All downloaded files will be saved to zip archive named "playlist_album", at the end you will be prompted to download this zip file. 
6. If necessary, you can download each file individually from file browser (see Files on the left)
7. When archive is downloaded, you can clean the files by running the last cell.
8. Be patient, downloading zip file takes time. You can check the progress in the output.

In [ ]:
#@title Video + Playlist Downloader { vertical-output: true }
url = "https://youtube.com/playlist?list=PLiN-7mukU_REPaZRXd62NKx1zoFXLnIRe" #@param {type:"string"}
# max_filesize_mb = 1000 #@param { type: 'slider', min: 1, max: 1000}
playlist_start = 1 #@param { type: 'number' }
playlist_end = 12 #@param { type: 'number' }

quality = "160K" #@param { type: "string" }
playlist_album = "Blade Runner" #@param { type: "string" }
playlist_artist = "Vangelis" #@param { type: "string" }
download_videos = False #@param { type: 'boolean' }

remove_everything = True #@param { type: 'boolean' }
download_full_playlist= True #@param { type: 'boolean' }
start_downloading_immediately = True #@param { type: 'boolean' }

title=playlist_album.replace(" ", "_") + ".zip"


!pip install yt-dlp
!pip install mutagen

if remove_everything:
    ! rm -rf *

import google

#Download videos?:###############

if download_videos:
    if download_full_playlist:
        ! yt-dlp {url} --format 18
    else:
        ! yt-dlp {url} --playlist-start {playlist_start} --playlist-end {playlist_end} --format 18
else:
    if download_full_playlist:
        ! yt-dlp {url} --ignore-errors --format bestaudio --extract-audio --audio-format mp3 --audio-quality {quality} --output "%(playlist_index)s - %(title)s.%(ext)s" --yes-playlist
    else:
        ! yt-dlp {url} --ignore-errors --format bestaudio --extract-audio --audio-format mp3 --audio-quality {quality} --output "%(playlist_index)s - %(title)s.%(ext)s" --playlist-start {playlist_start} --playlist-end {playlist_end} --yes-playlist
  


# TAGGING MP3s ###################################

from mutagen.mp3 import MP3  
from mutagen.easyid3 import EasyID3  
import mutagen.id3  
from mutagen.id3 import ID3, TIT2, TIT3, TALB, TPE1, TRCK, TYER  
  
import glob  
import numpy as np  


filez = glob.glob("*.mp3")  
  
for i in np.arange(0, len (filez)):  
	# extract the length of the directory  
	length_directory = len(filez[i].split("/"))  
	# extract the track number from the last element of the file path  
	tracknum = filez[i].split("/")[length_directory-1][0:2]  
	# mp3 name (with directory) from filez  
	song = filez[i]  
	# turn it into an mp3 object using the mutagen library  
	mp3file = MP3(song, ID3=EasyID3)  
	# set the album name  
	mp3file['album'] = [playlist_album]  
	# set the albumartist name  
	mp3file['albumartist'] = [playlist_artist]  
	# set the track number with the proper format  
	mp3file['tracknumber'] = str(tracknum) + '/' + str(len(filez))  
	# save the changes that we've made  
	mp3file.save()   

########### Zipping and Downloading #################
! zip {title} *

if start_downloading_immediately:
    google.colab.files.download(title)

In [ ]:
# Clean up (after downloading mp3s and or zip)
! rm -f *